In [1]:
import torch
import torch.onnx
from torch.onnx import OperatorExportTypes

ckpt_path = "/ckpt/last.ckpt"
checkpoint = torch.load(ckpt_path,map_location="cuda")

# def custom_atan2_symbolic(g, self, other):
#     return g.op("Atan2", self, other)

# # 注册自定义 symbolic 函数到指定的 opset 版本（这里用 16，你也可以尝试其他版本）
# torch.onnx.register_custom_op_symbolic('aten::atan2', custom_atan2_symbolic, 16)




# def natten1dqkrpb_symbolic(g, query, key, rpb, kernel_size, dilation):
#     """
#     将 NATTEN1DQKRPBFunction 映射到一个自定义 ONNX 节点 “NATTEN1DQKRPB”。
#     我们把 kernel_size、dilation 和 bias（根据 rpb 是否为 None）作为属性传递。
#     """
#     bias = 1 if rpb is not None else 0
#     return g.op("NATTEN1DQKRPB",
#                 query, key, rpb,
#                 kernel_size_i=kernel_size,
#                 dilation_i=dilation,
#                 bias_i=bias)

# import natten.functional as nf  # 假设 NATTEN1DQKRPBFunction 定义在这里

# # 给 NATTEN1DQKRPBFunction 添加 symbolic 方法
# nf.NATTEN1DQKRPBFunction.symbolic = staticmethod(natten1dqkrpb_symbolic)

# # 注意：这里注册的名称 '::NATTEN1DQKRPBFunction' 必须与你模型中调用的 Python operator 名称一致
# torch.onnx.register_custom_op_symbolic('natten::NATTEN1DQKRPBFunction', natten1dqkrpb_symbolic, 16)


import os
import sys


# 配置 planner 相关参数
onnx_output_path = "/home/bydguikong/yy_ws/PlanScope/onnx/model.onnx"

sys.path.append('/home/bydguikong/yy_ws/PlanScope/')
from src.planners.ml_planner_utils import global_trajectory_to_states, load_checkpoint
from nuplan.planning.training.modeling.torch_module_wrapper import TorchModuleWrapper
from src.models.pluto.scope_model import *

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
planner = PlanningModel(
    dim =  128,
    state_channel =  6,
    polygon_channel =  6,
    history_channel =  9,
    history_steps =  21,
    future_steps =  80,
    encoder_depth =  4,
    decoder_depth =  4,
    drop_path =  0.2,
    dropout =  0.1,
    num_heads =  4,             #
    num_modes =  12,            #
    use_ego_history =  False,
    state_attn_encoder =  True,
    state_dropout =  0.75,
    use_hidden_proj = False,     #
    cat_x = True,               #非常重要
    ref_free_traj =  False,     #
    recursive_decoder =  False,
    residual_decoder =  False,
    multihead_decoder =  False,
    independent_detokenizer =  False,
    mvn_loss =  False,
    wtd_with_history =  False,    
)
# print("planner = ",planner)
mk, uk = planner.load_state_dict(load_checkpoint(ckpt_path), strict=False)
planner.to(device)
planner.eval()  # 设置为评估模式

agent_num = 15
map_line_num = 200
object_num = 100


dummy_input = [
    # agent 部分
    torch.randn(1, agent_num, 21, 2, dtype= torch.float32, device=device),             # 0: agent.position
    torch.randn(1, agent_num, 21, dtype= torch.float32, device=device),                # 1: agent.heading
    torch.randn(1, agent_num, 21, 2, dtype= torch.float32, device=device),             # 2: agent.velocity
    torch.randn(1, agent_num, 21, 2, dtype= torch.float32, device=device),             # 3: agent.shape
    torch.randint(0, 3, (1, agent_num), dtype=torch.int8, device=device),              # 4: agent.category
    torch.randint(0, 2, (1, agent_num, 21), dtype=torch.bool, device=device),          # 5: agent.valid_mask

    # map 部分
    torch.randn(1, map_line_num, 3, 20, 2, dtype= torch.float32, device=device),       # 6: map.point_position
    torch.randn(1, map_line_num, 3, 20, 2, dtype= torch.float32, device=device),       # 7: map.point_vector
    torch.randn(1, map_line_num, 3, 20, dtype= torch.float32, device=device),          # 8: map.point_orientation
    torch.randn(1, map_line_num, 3, dtype= torch.float32, device=device),              # 9: map.polygon_center
    torch.randint(0, 3, (1, map_line_num), dtype=torch.int8, device=device),      # 10: map.polygon_type
    torch.randint(0, 2, (1, map_line_num), dtype=torch.bool, device=device),      # 11: map.polygon_on_route
    torch.randint(0, 4, (1, map_line_num), dtype=torch.int8, device=device),      # 12: map.polygon_tl_status
    torch.randint(0, 2, (1, map_line_num), dtype=torch.bool, device=device),      # 13: map.polygon_has_speed_limit
    torch.randn(1, map_line_num, dtype= torch.float32, device=device),            # 14: map.polygon_speed_limit
    torch.randint(0, 2, (1, map_line_num, 20), dtype=torch.bool, device=device),  # 15: map.valid_mask

    # reference_line 部分
    torch.randn(1, 1, 120, 2, dtype= torch.float32, device=device),             # 16: reference_line.position
    torch.randn(1, 1, 120, 2, dtype= torch.float32, device=device),             # 17: reference_line.vector
    torch.randn(1, 1, 120, dtype= torch.float32, device=device),                # 18: reference_line.orientation
    torch.randint(0, 2, (1, 1, 120), dtype=torch.bool, device=device),          # 19: reference_line.valid_mask

    # static_objects 部分
    torch.randn(1, object_num, 2, dtype= torch.float32, device=device),                 # 20: static_objects.position
    torch.randn(1, object_num, dtype= torch.float32, device=device),                    # 21: static_objects.heading
    torch.randn(1, object_num, 2, dtype= torch.float32, device=device),                 # 22: static_objects.shape
    torch.randint(1, 4, (1, object_num), dtype= torch.float32, device=device),          # 23: static_objects.category
    torch.randint(0, 2, (1, object_num), dtype=torch.bool, device=device),              # 24: static_objects.valid_mask

    # 其他部分
    torch.randn(1, 7, dtype= torch.float32, device=device),                     # 25: current_state
]
# dummy_input = [
#     # agent 部分
#     torch.randn(1, agent_num, 21, 2, dtype= torch.float32, device=device),             # 0: agent.position
#     torch.randn(1, agent_num, 21, dtype= torch.float32, device=device),                # 1: agent.heading
#     torch.randn(1, agent_num, 21, 2, dtype= torch.float32, device=device),             # 2: agent.velocity
#     torch.randn(1, agent_num, 21, 2, dtype= torch.float32, device=device),             # 3: agent.shape
#     torch.randint(0, 3, (1, agent_num), dtype=torch.int8, device=device),              # 4: agent.category
#     torch.randint(0, 2, (1, agent_num, 21), dtype=torch.bool, device=device),          # 5: agent.valid_mask
#     # torch.randn(1, agent_num, 0, 3, dtype= torch.float32, device=device),              # 6: agent.target

#     # map 部分
#     torch.randn(1, map_line_num, 3, 20, 2, dtype= torch.float32, device=device),         # 7: map.point_position
#     torch.randn(1, map_line_num, 3, 20, 2, dtype= torch.float32, device=device),         # 8: map.point_vector
#     torch.randn(1, map_line_num, 3, 20, dtype= torch.float32, device=device),            # 9: map.point_orientation
#     torch.randint(0, 3, (1, map_line_num, 3), dtype= torch.int8, device=device),         # 10: map.point_side
#     torch.randn(1, map_line_num, 3, dtype= torch.float32, device=device),                # 11: map.polygon_center
#     torch.randn(1, map_line_num, 2, dtype= torch.float32, device=device),                # 12: map.polygon_position
#     torch.randn(1, map_line_num, dtype= torch.float32, device=device),                   # 13: map.polygon_orientation
#     torch.randint(0, 3, (1, map_line_num), dtype=torch.int8, device=device),  # 14: map.polygon_type
#     torch.randint(0, 2, (1, map_line_num), dtype=torch.bool, device=device),  # 15: map.polygon_on_route
#     torch.randint(0, 4, (1, map_line_num), dtype=torch.int8, device=device),  # 16: map.polygon_tl_status
#     torch.randint(0, 2, (1, map_line_num), dtype=torch.bool, device=device),  # 17: map.polygon_has_speed_limit
#     torch.randn(1, map_line_num, dtype= torch.float32, device=device),            # 18: map.polygon_speed_limit
#     torch.randint(0, 67000, (1, map_line_num), dtype=torch.int32, device=device),  # 19: map.polygon_road_block_id
#     torch.randint(0, 2, (1, map_line_num, 20), dtype=torch.bool, device=device),  # 20: map.valid_mask

#     # reference_line 部分
#     torch.randn(1, 1, 120, 2, dtype= torch.float32, device=device),             # 21: reference_line.position
#     torch.randn(1, 1, 120, 2, dtype= torch.float32, device=device),             # 22: reference_line.vector
#     torch.randn(1, 1, 120, dtype= torch.float32, device=device),                # 23: reference_line.orientation
#     torch.randint(0, 2, (1, 1, 120), dtype=torch.bool, device=device),          # 24: reference_line.valid_mask
#     torch.randn(1, 1, 8, 2, dtype= torch.float32, device=device),               # 25: reference_line.future_projection

#     # static_objects 部分
#     torch.randn(1, object_num, 2, dtype= torch.float32, device=device),                 # 26: static_objects.position
#     torch.randn(1, object_num, dtype= torch.float32, device=device),                    # 27: static_objects.heading
#     torch.randn(1, object_num, 2, dtype= torch.float32, device=device),                 # 28: static_objects.shape
#     torch.randint(1, 4, (1, object_num), dtype= torch.float32, device=device),                    # 29: static_objects.category
#     torch.randint(0, 2, (1, object_num), dtype=torch.bool, device=device),              # 30: static_objects.valid_mask

#     # 其他部分
#     torch.randn(1, 7, dtype= torch.float32, device=device),                     # 31: current_state
#     torch.randn(1, 2, dtype= torch.float32, device=device),                     # 32: origin
#     torch.randn(1, dtype= torch.float32, device=device),                        # 33: angle
# ]

# 为方便 ONNX 导出，定义每个输入的名称（顺序与 dummy_input 一一对应）
input_names = [
    "agent.position", "agent.heading", "agent.velocity", "agent.shape",
    "agent.category", "agent.valid_mask", #"agent.target",
    "map.point_position", "map.point_vector", "map.point_orientation", #"map.point_side",
    "map.polygon_center", #"map.polygon_position", #"map.polygon_orientation",
    "map.polygon_type", "map.polygon_on_route", "map.polygon_tl_status",
    "map.polygon_has_speed_limit", "map.polygon_speed_limit", #"map.polygon_road_block_id",
    "map.valid_mask",
    "reference_line.position", "reference_line.vector", "reference_line.orientation",
    "reference_line.valid_mask", #"reference_line.future_projection",
    "static_objects.position", "static_objects.heading", "static_objects.shape",
    "static_objects.category", "static_objects.valid_mask",
    "current_state", #"origin", "angle"
]
# input_names = [
#     "agent.position", "agent.heading", "agent.velocity", "agent.shape",
#     "agent.category", "agent.valid_mask", "agent.target",
#     "map.point_position", "map.point_vector", "map.point_orientation", "map.point_side",
#     "map.polygon_center", "map.polygon_position", "map.polygon_orientation",
#     "map.polygon_type", "map.polygon_on_route", "map.polygon_tl_status",
#     "map.polygon_has_speed_limit", "map.polygon_speed_limit", "map.polygon_road_block_id",
#     "map.valid_mask",
#     "reference_line.position", "reference_line.vector", "reference_line.orientation",
#     "reference_line.valid_mask", "reference_line.future_projection",
#     "static_objects.position", "static_objects.heading", "static_objects.shape",
#     "static_objects.category", "static_objects.valid_mask",
#     "current_state", "origin", "angle"
# ]

# 3. 定义 dynamic_axes
# 这里只设置可能动态变化的轴：batch、agent数量、地图元素数量、参考线采样点数、静态物体数量等
dynamic_axes = {
    "agent.position": {0: "batch_size", 1: "num_agents"},
    "agent.heading": {0: "batch_size", 1: "num_agents"},
    "agent.velocity": {0: "batch_size", 1: "num_agents"},
    "agent.shape": {0: "batch_size", 1: "num_agents"},
    "agent.category": {0: "batch_size", 1: "num_agents"},
    "agent.valid_mask": {0: "batch_size", 1: "num_agents"},
    "agent.target": {0: "batch_size", 1: "num_agents"},

    "map.point_position": {0: "batch_size", 1: "num_map_elements"},
    "map.point_vector": {0: "batch_size", 1: "num_map_elements"},
    "map.point_orientation": {0: "batch_size", 1: "num_map_elements"},
    "map.point_side": {0: "batch_size", 1: "num_map_elements"},
    "map.polygon_center": {0: "batch_size", 1: "num_map_elements"},
    "map.polygon_position": {0: "batch_size", 1: "num_map_elements"},
    "map.polygon_orientation": {0: "batch_size", 1: "num_map_elements"},
    "map.polygon_type": {0: "batch_size", 1: "num_map_elements"},
    "map.polygon_on_route": {0: "batch_size", 1: "num_map_elements"},
    "map.polygon_tl_status": {0: "batch_size", 1: "num_map_elements"},
    "map.polygon_has_speed_limit": {0: "batch_size", 1: "num_map_elements"},
    "map.polygon_speed_limit": {0: "batch_size", 1: "num_map_elements"},
    "map.polygon_road_block_id": {0: "batch_size", 1: "num_map_elements"},
    "map.valid_mask": {0: "batch_size", 1: "num_map_elements"},

    "reference_line.position": {0: "batch_size"},
    "reference_line.vector": {0: "batch_size"},
    "reference_line.orientation": {0: "batch_size"},
    "reference_line.valid_mask": {0: "batch_size"},
    "reference_line.future_projection": {0: "batch_size"},

    "static_objects.position": {0: "batch_size", 1: "num_static_objects"},
    "static_objects.heading": {0: "batch_size", 1: "num_static_objects"},
    "static_objects.shape": {0: "batch_size", 1: "num_static_objects"},
    "static_objects.category": {0: "batch_size", 1: "num_static_objects"},
    "static_objects.valid_mask": {0: "batch_size", 1: "num_static_objects"},

    "current_state": {0: "batch_size"},
    "origin": {0: "batch_size"},
    "angle": {0: "batch_size"}
}
# dynamic_axes = {
#     "agent.position": {0: "batch_size"},
#     "agent.heading": {0: "batch_size"},
#     "agent.velocity": {0: "batch_size"},
#     "agent.shape": {0: "batch_size"},
#     "agent.category": {0: "batch_size"},
#     "agent.valid_mask": {0: "batch_size"},
#     "agent.target": {0: "batch_size"},

#     "map.point_position": {0: "batch_size"},
#     "map.point_vector": {0: "batch_size"},
#     "map.point_orientation": {0: "batch_size"},
#     "map.point_side": {0: "batch_size"},
#     "map.polygon_center": {0: "batch_size"},
#     "map.polygon_position": {0: "batch_size"},
#     "map.polygon_orientation": {0: "batch_size"},
#     "map.polygon_type": {0: "batch_size"},
#     "map.polygon_on_route": {0: "batch_size"},
#     "map.polygon_tl_status": {0: "batch_size"},
#     "map.polygon_has_speed_limit": {0: "batch_size"},
#     "map.polygon_speed_limit": {0: "batch_size"},
#     "map.polygon_road_block_id": {0: "batch_size"},
#     "map.valid_mask": {0: "batch_size"},

#     "reference_line.position": {0: "batch_size"},
#     "reference_line.vector": {0: "batch_size"},
#     "reference_line.orientation": {0: "batch_size"},
#     "reference_line.valid_mask": {0: "batch_size"},
#     "reference_line.future_projection": {0: "batch_size"},

#     "static_objects.position": {0: "batch_size"},
#     "static_objects.heading": {0: "batch_size"},
#     "static_objects.shape": {0: "batch_size"},
#     "static_objects.category": {0: "batch_size"},
#     "static_objects.valid_mask": {0: "batch_size"},

#     "current_state": {0: "batch_size"},
#     "origin": {0: "batch_size"},
#     "angle": {0: "batch_size"}
# }
print("device = ",device)
# # 导出ONNX模型
# scripted_model = torch.jit.script(planner)
torch.onnx.export(
    # scripted_model,          # 要导出的模型
    planner,          # 要导出的模型
    dummy_input,                     # 用于导出模型的输入
    onnx_output_path,                # 输出文件路径
    opset_version=11,                 # 选择适合的ONNX版本
    input_names=input_names,   # 扁平化后的输入名称列表
    # output_names=["output"],  # 
    output_names=["T_trajectory","S_probability","P_prediction","output_prediction","output_trajectory","candidate_trajectories"],# 
    dynamic_axes=dynamic_axes,  # 动态轴信息
    verbose=False,  # 是否打印详细信息
    do_constant_folding=True,     # 启用常量折叠
    operator_export_type=OperatorExportTypes.ONNX, #使用标准算子导出，避免 ATEN 或 PythonOp
    # operator_export_type=torch.onnx.OperatorExportTypes.ONNX_ATEN_FALLBACK,
    # operator_export_type=OperatorExportTypes.ONNX_FALLTHROUGH,
)

print(f"ONNX model saved to {onnx_output_path}")


FileNotFoundError: [Errno 2] No such file or directory: '/home/bydguikong/yy_ws/PlanScope/checkpoints/last.ckpt'

In [ ]:
!python -m onnxsim ./onnx/model.onnx ./onnx/simplified_model.onnx

In [ ]:
import netron

netron.start(R"/home/bydguikong/yy_ws/PlanScope/onnx/model.onnx")
netron.start(R"/home/bydguikong/yy_ws/PlanScope/onnx/simplified_model.onnx")
# netron.start(R"/home/bydguikong/yy_ws/PlanScope/onnx/simplified_model.onnx")